# Willkommen zur Einführung in Medizinische Bildverarbeitung
Prof. Dr. Thomas Schrader

Technische Hochschule Brandenburg

## Was wollen wir in diesem Workshop machen?
Rausfinden, 
- was digitale Bilder sind: Wie werden digitale Bilder im Computer dargestellt und verarbeitet?
- medizinische Bilder so besonders macht (in mehrfacher Hinsicht)
- wie wir Bilder in der Qualität verändern können (heller, dunkler, Kontrast)
- wie wir in medizinischen Bildern interessante Regionen markieren und vermessen können

## Was wir für die Arbeit brauchen
Python ist eine OpenSource Programmiersprache. Mit Google Colab ist es möglich, Python online auszuführen. Dazu wird eine beliebte Programmierumgebung verwendet: Jupyter-Notebook. 

Jupyter-Notebook vereint
- Dokumentation
- Quellcode und 
- Darstellung der Ergebnisse 
in einem Dokument. Es kann problemlos ausgetauscht und ausgedruckt werden

Mit Python arbeiten zig Tausend Entwickler:innen. Sie haben Bibliotheken zur Verfügung gestellt, die es erlauben, komplizierte Operationen mit einer Zeile Quellcode auszuführen, weil sie die Funktionen dafür geschrieben haben. Für Python ist das sehr gut und manchmal auch etwas komplizert: Für die Bildverarbeitung gibt es viele verschiedene Bibliotheken, die alles etwas unterschiedlich angesprochen werden und auch die Bilder intern etwas unterschiedlich zwischenspeichern. 

Wir arbeiten hier mit unter anderem mit der wohl bekanntesten Bibliothek für Computer Vision und Bildverarbeitung: OpenCV.

Wir brauchen allerdings auch eine Bibliothek, die medizinische Bilder öffnen kann: pydicom

Zum Darstellen von Bildern und Diagrammen gibt es die sehr häufig verwendete Bibliothek matplotlib.


Diese Bibliotheken müssen in den Arbeitsspeicher geladen werden.

## Laden der notwendigen Bibliotheken


In [ ]:
import os
!pip install opencv-python
import cv2
import numpy as np
! pip install pydicom
import pydicom as dc
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s eta 0:00:00


Die Beispielbilder liegen in einem separaten Verzeichnis

In [ ]:
fDir ="/content/drive/MyDrive/Colab Notebooks/MedDIP.Starter/images/"

## Laden eines 'normalen' Bildes
... wie es sonst aus jedem Fotoapparat kommt

In [ ]:
fName = "1.3.6.1.4.1.9590.100.1.2.111673900911394832129091490821952505416.tif"
img = cv2.imread(fDir+fName)

In [ ]:
img[1:3,1:3,1]

In [ ]:
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(img)

In [ ]:
r, g, b = cv2.split(img)
plt.figure(figsize=[20,10])
plt.subplot(131)
plt.imshow(r,'gray')
plt.subplot(132)
plt.imshow(g,'gray')
plt.subplot(133)
plt.imshow(b,'gray')

In [ ]:
imgN = cv2.merge([r+10,g,b-100])
plt.imshow(imgN)

In [ ]:
print(imgN.shape)

In [ ]:
imgG = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
plt.imshow(imgG,'gray')

## Eigenschaften des Bildes: Verteilung der Grauwerte (bzw. RGB-Werte)
Die Verteilung der Grauwerte (bzw. ggf. der Farbwerte) sind interessant, weil sie wichtige Informationen über die Helligkeit und den Kontrast liefern. 

In [ ]:
gHist = cv2.calcHist(imgG, [0], None, [256],[0,256])
plt.figure(figsize = [10,5])
plt.plot(gHist)
plt.title('Histogram')
plt.ylabel('Häufigkeit')
plt.xlabel('Grauwert')

In [ ]:
rHist = cv2.calcHist(img, [0], None, [256],[0,256])
gHist = cv2.calcHist(img, [1], None, [256],[0,256])
bHist = cv2.calcHist(img, [2], None, [256],[0,256])

In [ ]:
plt.figure(figsize = [10,5])
plt.plot(rHist,'g')
plt.plot(gHist,'r')
plt.plot(bHist,'b')
plt.title('Histogram')
plt.ylabel('Häufigkeit')
plt.xlabel('Grauwert')

# DICOM 
## Was ist DICOM
DICOM heisst ausgesprochen: Digital Imaging and Communications in Medicine und ist DER Standard für alle Medien, die im medizinischen Kontext verwendet werden. 
Begonnen hat der Standard natürlich mit Bildern, mit CT-Bildern, das waren die ersten digitalen Bilder in der Radiologie, die systematisch erzeugt wurden. Dann kam weiteres hinzu Ultraschall, normales Röntgen, MRT etc. 
Was ist so besonders: 
- Alle Bilder werden mit wichtigen Zusatzinformationen versehen - den Metadaten
- Auch wie die Bilder gespeichert werden, wie aufgerufen und wie sie erzeugt werden, ist alles im Standard geregelt

In [ ]:
fName = 'brain_009.dcm'
print(fDir)
fileList  = os.listdir(fDir)
for item in fileList: 
  print(item) 

In [ ]:
dcImg = dc.dcmread(fDir+fName)

plt.figure()
plt.imshow(dcImg.pixel_array, cmap = 'gray')

Jetzt schauen wir uns mal an, was das für Pixelwerte sind, denn die sind nämlich etwas anders.

In [ ]:
plt.figure()
bins = 200
hbins0, cl0 = np.histogram(dcImg.pixel_array, bins = bins)
print(cl0.shape, hbins0.shape)
plt.plot(cl0[0:len(cl0)-1], hbins0)



Eine Besonderheit: Hounsfield



In [ ]:
dcImgHU = dcImg.pixel_array*dcImg.RescaleSlope + dcImg.RescaleIntercept

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(dcImgHU)

In [ ]:
hbins1, cl1 = np.histogram(dcImgHU, bins = bins)
plt.figure()
plt.plot(cl1[0:len(cl1)-1],hbins1)
plt.plot(cl0[0:len(cl0)-1],hbins0)